# 3D Scene Reconstruction with Outlier Detection (GPU Optimized Approach)
In this notebook, we'll develop a solution for reconstructing 3D scenes from image collections while identifying and filtering out unrelated images. Our approach involves detecting which images belong to the same scenes and which are outliers, followed by camera pose estimation for the images that belong togther.

## Setup and GPU Optimization

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import networkx as nx
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import warnings
import torch
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Define paths
TRAIN_DIR = "/kaggle/input/image-matching-challenge-2025/train/"  # Path to training data
TEST_DIR = "/kaggle/input/image-matching-challenge-2025/test/"    # Path to test data
OUTPUT_FILE = "submission.csv"  # Path for output file

# Check GPU availability and set up CUDA for GPU acceleration
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True  # Optimize CUDA performance
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    # Print GPU memory info
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Enable OpenCV GPU acceleration if available
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    print("OpenCV CUDA acceleration enabled")
else:
    print("OpenCV CUDA acceleration not available")

# Memory management function
def free_memory():
    """Free GPU memory and call garbage collector."""
    torch.cuda.empty_cache()
    gc.collect()

## Read Training Data and Define Parameters

In [ ]:
# Read training labels and thresholds from CSV files
print("\nReading training CSV files...")

def read_training_csvs():
    """Read training CSV files with ground truth scene labels.
    
    Returns:
        Dictionary with training CSV data
    """
    csv_data = {
        'labels': None,
        'thresholds': None
    }
    
    # Look for train_labels.csv
    labels_path = '/kaggle/input/image-matching-challenge-2025/train_labels.csv'
    if Path(labels_path).exists():
        try:
            labels_df = pd.read_csv(labels_path)
            print(f"Found train_labels.csv with {len(labels_df)} rows")
            csv_data['labels'] = labels_df
        except Exception as e:
            print(f"Error reading train_labels.csv: {e}")
    else:
        print("train_labels.csv not found")
        
    # Look for train_thresholds.csv
    thresholds_path = '/kaggle/input/image-matching-challenge-2025/train_thresholds.csv'
    if Path(thresholds_path).exists():
        try:
            thresholds_df = pd.read_csv(thresholds_path)
            print(f"Found train_thresholds.csv with {len(thresholds_df)} rows")
            csv_data['thresholds'] = thresholds_df
        except Exception as e:
            print(f"Error reading train_thresholds.csv: {e}")
    else:
        print("train_thresholds.csv not found")
        
    return csv_data

# Read training CSVs
training_csvs = read_training_csvs()

# Set feature matching parameters - optimized based on ground truth data
FEATURE_MATCHING_PARAMS = {
    'ratio_test': 0.75,  # Lowe's ratio test threshold
    'min_inliers': 15,   # Minimum number of inliers required
    'inlier_ratio': 0.3  # Minimum ratio of inliers to matches
}

## Analyze Training Data


In [ ]:
# If we found the training CSV files, use them to extract ground truth information
if training_csvs['labels'] is not None:
    print("\nExtracting ground truth information from training CSV files...")
    
    # Extract ground truth scene assignments
    labels_df = training_csvs['labels']
    
    # Get unique datasets and count their scenes
    dataset_scenes = labels_df.groupby('dataset')['scene'].nunique()
    
    # Build training structure from CSV data
    training_structure = {}
    
    for dataset, scene_count in dataset_scenes.items():
        # Get dataset images
        dataset_images = labels_df[labels_df['dataset'] == dataset]
        
        # Count images per scene
        scene_counts = dataset_images.groupby('scene').size()
        
        # Check for outliers
        has_outliers = 'outliers' in scene_counts.index
        outlier_count = scene_counts.get('outliers', 0)
        
        # Calculate total images and outlier percentage
        total_images = dataset_images.shape[0]
        outlier_percentage = (outlier_count / total_images) * 100 if total_images > 0 else 0
        
        # Add to training structure
        training_structure[dataset] = {
            'path': Path(TRAIN_DIR) / dataset,
            'expected_scene_count': scene_count - (1 if has_outliers else 0),  # Don't count outliers as a scene
            'scene_counts': scene_counts.to_dict(),
            'total_images': total_images,
            'outlier_count': outlier_count,
            'outlier_percentage': outlier_percentage
        }
    
    # Print training structure information
    print("\nTraining Structure from CSV:")
    for dataset, info in training_structure.items():
        scene_count = info['expected_scene_count']
        print(f"  {dataset}: {scene_count} scenes, {info['total_images']} images, {info['outlier_count']} outliers ({info['outlier_percentage']:.1f}%)")
    
    # Calculate training statistics
    scene_counts = [info['expected_scene_count'] for _, info in training_structure.items()]
    image_per_scene = []
    outlier_percentages = [info['outlier_percentage'] for _, info in training_structure.items()]
    
    # Calculate average images per scene (excluding outliers)
    for dataset, info in training_structure.items():
        scene_image_counts = {k: v for k, v in info['scene_counts'].items() if k != 'outliers'}
        if scene_image_counts:
            image_per_scene.extend(scene_image_counts.values())
    
    training_stats = {
        'avg_scenes_per_dataset': np.mean(scene_counts) if scene_counts else 2.0,
        'avg_images_per_scene': np.mean(image_per_scene) if image_per_scene else 15.0,
        'avg_outlier_percentage': np.mean(outlier_percentages) if outlier_percentages else 10.0,
        'outlier_datasets_percentage': sum(1 for info in training_structure.values() if info['outlier_count'] > 0) / len(training_structure) * 100 if training_structure else 50.0
    }
    
    print("\nTraining Statistics from CSV:")
    print(f"  Average scenes per dataset: {training_stats['avg_scenes_per_dataset']:.2f}")
    print(f"  Average images per scene: {training_stats['avg_images_per_scene']:.2f}")
    print(f"  Average outlier percentage: {training_stats['avg_outlier_percentage']:.2f}%")
    print(f"  Datasets with outliers: {training_stats['outlier_datasets_percentage']:.2f}%")
    
    # Check if thresholds are available
    if training_csvs['thresholds'] is not None:
        thresholds_df = training_csvs['thresholds']
        print("\nThreshold information available for scenes")
else:
    print("No training CSV files found. Using default parameters.")
    # Use default parameters
    training_structure = {}
    training_stats = {
        'avg_scenes_per_dataset': 2.31,  # Based on previous analysis
        'avg_images_per_scene': 60.77,
        'avg_outlier_percentage': 5.85,
        'outlier_datasets_percentage': 30.77
    }

print("\nUsing Feature Matching Parameters:")
for param, value in FEATURE_MATCHING_PARAMS.items():
    print(f"  {param}: {value}")

# Set up test dataset info
test_dataset_info = {}
for dataset_path in Path(TEST_DIR).glob('*'):
    if dataset_path.is_dir():
        dataset_name = dataset_path.name
        
        # If we have this dataset in training, use its information
        if dataset_name in training_structure:
            expected_scene_count = training_structure[dataset_name]['expected_scene_count']
            print(f"Using scene count from training for {dataset_name}: {expected_scene_count} scenes")
            
            test_dataset_info[dataset_name] = {
                'path': dataset_path,
                'expected_scene_count': expected_scene_count
            }
        else:
            # No matching training data, use the average
            avg_scenes = int(round(training_stats['avg_scenes_per_dataset']))
            test_dataset_info[dataset_name] = {
                'path': dataset_path,
                'expected_scene_count': avg_scenes
            }
            print(f"No training data for {dataset_name}, using average: {avg_scenes} scenes")

## GPU-Accelerated Feature Extraction and Matching

In [ ]:
def extract_features_gpu(image_path):
    """Extract SIFT features from an image using GPU acceleration if available.
    
    Args:
        image_path: Path to the image file
        
    Returns:
        Tuple of (keypoints, descriptors, image_dimensions)
    """
    # Read image
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None, None
    
    # Use CUDA SIFT if available
    if hasattr(cv2, 'cuda_SIFT'):
        # Convert image to GPU
        gpu_img = cv2.cuda_GpuMat()
        gpu_img.upload(img)
        
        # Extract features
        sift_gpu = cv2.cuda_SIFT.create(nfeatures=2000)
        keypoints, descriptors = sift_gpu.detectAndCompute(gpu_img, None)
    else:
        # Use CPU SIFT
        sift = cv2.SIFT_create(nfeatures=2000)
        keypoints, descriptors = sift.detectAndCompute(img, None)
    
    return keypoints, descriptors, img.shape

def process_dataset_features(dataset_path, max_workers=4):
    """Process all images in a dataset and extract features using parallel processing.
    
    Args:
        dataset_path: Path to the dataset directory
        max_workers: Maximum number of parallel workers
        
    Returns:
        Dictionary mapping image names to their features
    """
    features_dict = {}
    
    # Get all image files (only .png for this dataset)
    image_files = list(dataset_path.glob('*.png'))
    
    # Define a worker function for parallel processing
    def process_image(img_path):
        keypoints, descriptors, dimensions = extract_features_gpu(img_path)
        if keypoints is not None and descriptors is not None and len(keypoints) > 0:
            return img_path.name, {
                'keypoints': keypoints,
                'descriptors': descriptors,
                'dimensions': dimensions
            }
        return None, None
    
    # Process images in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_image, img_path) for img_path in image_files]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Extracting features"):
            img_name, features = future.result()
            if img_name is not None:
                features_dict[img_name] = features
                features_dict[img_name]['path'] = dataset_path / img_name
    
    # Free memory after feature extraction
    free_memory()
    return features_dict

def match_features_gpu(desc1, desc2, kp1, kp2, params=None):
    """Match features between two images using ratio test and geometric verification.
    GPU accelerated when possible.
    
    Args:
        desc1, desc2: Feature descriptors from both images
        kp1, kp2: Keypoints from both images
        params: Dictionary with matching parameters
        
    Returns:
        Tuple of (matches, inlier_count, is_geometrically_consistent)
    """
    if params is None:
        params = FEATURE_MATCHING_PARAMS
    
    # Extract parameters
    ratio = params['ratio_test']
    min_inliers = params['min_inliers']
    inlier_ratio = params['inlier_ratio']
    
    # Handle empty descriptors
    if desc1 is None or desc2 is None or len(desc1) < 8 or len(desc2) < 8:
        return [], 0, False
    
    try:
        # Check if CUDA BFMatcher is available
        if hasattr(cv2, 'cuda_DescriptorMatcher_createBFMatcher'):
            # Upload descriptors to GPU
            gpu_desc1 = cv2.cuda_GpuMat()
            gpu_desc2 = cv2.cuda_GpuMat()
            gpu_desc1.upload(np.ascontiguousarray(desc1, dtype=np.float32))
            gpu_desc2.upload(np.ascontiguousarray(desc2, dtype=np.float32))
            
            # Create GPU matcher
            matcher = cv2.cuda_DescriptorMatcher_createBFMatcher(cv2.NORM_L2)
            gpu_matches = matcher.knnMatch(gpu_desc1, gpu_desc2, k=2)
            
            # Apply ratio test
            good_matches = [m for m, n in gpu_matches if m.distance < ratio * n.distance]
        else:
            # Use FLANN matcher on CPU
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            search_params = dict(checks=50)
            
            flann = cv2.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(desc1, desc2, k=2)
            
            # Apply ratio test to find good matches
            good_matches = []
            for i, pair in enumerate(matches):
                if len(pair) == 2:  # Ensure we have 2 matches
                    m, n = pair
                    if m.distance < ratio * n.distance:
                        good_matches.append(m)
        
        # Perform geometric verification if we have enough matches
        is_consistent = False
        inlier_count = 0
        
        if len(good_matches) >= 8:
            # Extract matched keypoints
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            
            # Find fundamental matrix
            F, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC, 3.0)
            
            if F is not None and mask is not None:
                inliers = mask.ravel() == 1
                inlier_count = np.sum(inliers)
                
                # Consider match geometrically consistent if enough inliers
                is_consistent = inlier_count >= max(min_inliers, inlier_ratio * len(good_matches))
    
    except Exception as e:
        # Handle potential errors in feature matching
        return [], 0, False
        
    return good_matches, inlier_count, is_consistent

def build_match_graph(features_dict, batch_size=100):
    """Build a graph representing image matches using batched processing.
    
    Args:
        features_dict: Dictionary of image features
        batch_size: Number of edges to process in each batch
        
    Returns:
        NetworkX graph where nodes are images and edges represent matches
    """
    # Create graph
    G = nx.Graph()
    
    # Add nodes for each image
    for img_name in features_dict.keys():
        G.add_node(img_name)
    
    # Create all image pairs for matching
    image_names = list(features_dict.keys())
    n = len(image_names)
    
    if n <= 1:  # Only proceed if we have at least 2 images
        return G
    
    # Create all pairs for processing
    pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    total_pairs = len(pairs)
    
    # Process in batches to avoid memory issues
    for batch_start in tqdm(range(0, total_pairs, batch_size), desc="Building match graph"):
        batch_end = min(batch_start + batch_size, total_pairs)
        batch_pairs = pairs[batch_start:batch_end]
        
        for i, j in batch_pairs:
            img1_name = image_names[i]
            img2_name = image_names[j]
            
            kp1 = features_dict[img1_name]['keypoints']
            desc1 = features_dict[img1_name]['descriptors']
            kp2 = features_dict[img2_name]['keypoints']
            desc2 = features_dict[img2_name]['descriptors']
            
            # Match features and verify
            _, inlier_count, is_consistent = match_features_gpu(desc1, desc2, kp1, kp2)
            
            # Add edge if geometrically consistent
            if is_consistent:
                G.add_edge(img1_name, img2_name, weight=inlier_count)
        
        # Free memory after each batch
        if batch_end % (batch_size * 10) == 0:
            free_memory()
    
    return G

## Efficient Scene Clustering

In [ ]:
def cluster_scenes(graph, expected_scene_count=None):
    """Cluster images into scenes based on the match graph.
    
    Args:
        graph: NetworkX graph of image matches
        expected_scene_count: Optional expected number of scenes
        
    Returns:
        Tuple of (scenes, outliers) where:
            scenes: List of lists, each containing image names in a scene
            outliers: List of image names identified as outliers
    """
    # Get graph nodes
    image_names = list(graph.nodes())
    
    # If graph is empty or has only one node, return early
    if len(image_names) <= 1:
        if len(image_names) == 1:
            return [], image_names  # Single image is an outlier
        else:
            return [], []  # Empty graph
    
    # If graph has no edges, all images are outliers
    if not graph.edges():
        return [], image_names
    
    # Analyze graph connectivity
    connected_components = list(nx.connected_components(graph))
    
    # If we have multiple connected components, use them as initial clusters
    if len(connected_components) > 1:
        scenes = []
        outliers = []
        
        for component in connected_components:
            component_list = list(component)
            if len(component_list) >= 3:
                # This is a potential scene
                scenes.append(component_list)
            else:
                # Too small to be a scene
                outliers.extend(component_list)
                
        return scenes, outliers
    
    # Get graph adjacency matrix for clustering
    adj_matrix = nx.to_numpy_array(graph)
    
    # Use spectral clustering if we have expected scene count and more than one scene
    if expected_scene_count is not None and expected_scene_count > 1:
        try:
            from sklearn.cluster import SpectralClustering
            
            # Create affinity matrix from adjacency (higher weight = higher affinity)
            max_weight = np.max(adj_matrix) if np.max(adj_matrix) > 0 else 1
            affinity_matrix = adj_matrix / max_weight
            
            # Apply spectral clustering
            clustering = SpectralClustering(
                n_clusters=expected_scene_count,
                affinity='precomputed',
                random_state=42
            )
            
            labels = clustering.fit_predict(affinity_matrix)
        except Exception:
            # Fall back to DBSCAN
            expected_scene_count = None
    
    # Use DBSCAN if no expected scene count or spectral clustering failed
    if expected_scene_count is None:
        # Convert to distance matrix (higher weight = lower distance)
        max_weight = np.max(adj_matrix) if np.max(adj_matrix) > 0 else 1
        dist_matrix = 1 - (adj_matrix / max_weight)
        np.fill_diagonal(dist_matrix, 0)  # Zero distance to self
        
        # Apply DBSCAN with best parameters from previous analysis
        clustering = DBSCAN(eps=0.6, min_samples=3, metric='precomputed')
        labels = clustering.fit_predict(dist_matrix)
    
    # Group images by cluster
    scenes = []
    outliers = []
    
    # Process each unique label
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:  # DBSCAN outliers (or not assigned in spectral clustering)
            outlier_indices = np.where(labels == -1)[0]
            outliers.extend([image_names[i] for i in outlier_indices])
        else:
            # Get cluster members
            cluster_indices = np.where(labels == label)[0]
            cluster_names = [image_names[i] for i in cluster_indices]
            
            # Check if the cluster forms a connected component in the graph
            subgraph = graph.subgraph(cluster_names)
            
            if len(cluster_names) >= 3:  # Minimum cluster size
                if nx.is_connected(subgraph):
                    scenes.append(cluster_names)
                else:
                    # If not connected, split into connected components
                    sub_components = list(nx.connected_components(subgraph))
                    for component in sub_components:
                        comp_list = list(component)
                        if len(comp_list) >= 3:
                            scenes.append(comp_list)
                        else:
                            outliers.extend(comp_list)
            else:
                # If too small, treat as outliers
                outliers.extend(cluster_names)
    
    # Add disconnected nodes (not in any edge) to outliers
    connected_nodes = set()
    for edge in graph.edges():
        connected_nodes.add(edge[0])
        connected_nodes.add(edge[1])
    
    disconnected = set(graph.nodes()) - connected_nodes
    outliers.extend(disconnected)
    
    return scenes, outliers

## Optimized Camera Pose Estimation

In [ ]:
def estimate_poses(scene_images, features_dict):
    """Estimate camera poses for a scene using Structure from Motion approach.
    
    Args:
        scene_images: List of image names in the scene
        features_dict: Dictionary of image features
        
    Returns:
        Dictionary mapping image names to (R, T) pairs
    """
    # Initialize poses dictionary
    poses = {}
    
    # Set first camera as reference (identity rotation, zero translation)
    if scene_images:
        poses[scene_images[0]] = (np.eye(3), np.zeros(3))
    
    # If only one image, we're done
    if len(scene_images) == 1:
        return poses
    
    # Build a spanning tree of matches
    G = nx.Graph()
    for img_name in scene_images:
        G.add_node(img_name)
    
    # Add edges with weights based on match quality
    for i, img1 in enumerate(scene_images):
        kp1 = features_dict[img1]['keypoints']
        desc1 = features_dict[img1]['descriptors']
        
        for j in range(i+1, len(scene_images)):
            img2 = scene_images[j]
            kp2 = features_dict[img2]['keypoints']
            desc2 = features_dict[img2]['descriptors']
            
            _, inlier_count, is_consistent = match_features_gpu(desc1, desc2, kp1, kp2)
            
            if is_consistent:
                G.add_edge(img1, img2, weight=inlier_count)
    
    # Find maximum spanning tree to avoid loops
    if len(G.edges()) > 0:
        try:
            mst = nx.maximum_spanning_tree(G, weight='weight')
            
            # Add remaining images using breadth-first search from reference camera
            queue = [scene_images[0]]
            visited = {scene_images[0]}
            
            while queue and len(poses) < len(scene_images):
                current = queue.pop(0)
                
                # Process neighbors
                for neighbor in mst.neighbors(current):
                    if neighbor not in visited:
                        # Get feature matches
                        kp1 = features_dict[current]['keypoints']
                        desc1 = features_dict[current]['descriptors']
                        kp2 = features_dict[neighbor]['keypoints']
                        desc2 = features_dict[neighbor]['descriptors']
                        
                        matches, _, _ = match_features_gpu(desc1, desc2, kp1, kp2)
                        
                        if len(matches) >= 8:
                            # Extract matched points
                            pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
                            pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
                            
                            # Estimate essential matrix
                            focal = max(features_dict[current]['dimensions']) / 2
                            pp = (features_dict[current]['dimensions'][1] / 2, 
                                features_dict[current]['dimensions'][0] / 2)
                            
                            E, mask = cv2.findEssentialMat(pts1, pts2, focal=focal, pp=pp, method=cv2.FM_RANSAC)
                            
                            if E is not None:
                                # Recover relative pose
                                _, R, t, _ = cv2.recoverPose(E, pts1, pts2, focal=focal, pp=pp)
                                
                                # Get current pose
                                R_current, t_current = poses[current]
                                
                                # Calculate neighbor pose based on relative pose
                                R_neighbor = R_current @ R.T
                                t_neighbor = t_current - R_neighbor @ t.ravel()
                                
                                # Store pose
                                poses[neighbor] = (R_neighbor, t_neighbor)
                        
                        # Mark as visited and add to queue
                        visited.add(neighbor)
                        queue.append(neighbor)
        except Exception as e:
            pass
    
    # For images without poses (unregistered), add them but mark with None
    for img_name in scene_images:
        if img_name not in poses:
            poses[img_name] = (None, None)
    
    return poses

## Process Test Datasets with Memory Management

In [ ]:
def format_matrix_for_submission(matrix):
    """Format a matrix as a semicolon-separated string."""
    if matrix is None:
        return ";".join(["nan"] * 9)
    return ";".join([str(float(x)) for x in matrix.flatten()])

def format_vector_for_submission(vector):
    """Format a vector as a semicolon-separated string."""
    if vector is None:
        return ";".join(["nan"] * 3)
    return ";".join([str(float(x)) for x in vector])

def process_test_dataset(dataset_path, dataset_name, training_structure, test_dataset_info):
    """Process a test dataset to identify scenes and estimate poses.
    
    Args:
        dataset_path: Path to test dataset
        dataset_name: Name of the dataset
        training_structure: Structure information from training data
        test_dataset_info: Additional analysis info for test datasets
        
    Returns:
        Dictionary with scenes, outliers, and poses
    """
    print(f"  Processing dataset '{dataset_name}'...")
    
    # Extract features from test dataset
    features_dict = process_dataset_features(dataset_path)
    
    if not features_dict:
        print(f"  Warning: No features extracted from {dataset_name}")
        return {
            'scenes': [],
            'outliers': [img.name for img in dataset_path.glob('*.png')],
            'poses': {}
        }
    
    print(f"  Extracted features from {len(features_dict)} images")
    
    # Build match graph
    match_graph = build_match_graph(features_dict)
    
    print(f"  Match graph has {match_graph.number_of_nodes()} nodes and {match_graph.number_of_edges()} edges")
    
    # Get expected scene count from test_dataset_info if available
    expected_scene_count = None
    if dataset_name in test_dataset_info:
        expected_scene_count = test_dataset_info[dataset_name].get('expected_scene_count')
        print(f"  Using expected scene count from analysis: {expected_scene_count}")
    
    # Cluster into scenes
    print(f"  Clustering scenes...")
    scenes, outliers = cluster_scenes(match_graph, expected_scene_count)
    
    print(f"  Identified {len(scenes)} scenes and {len(outliers)} outliers")
    
    # Estimate poses for each scene
    all_poses = {}
    
    for i, scene in enumerate(scenes):
        print(f"  Estimating poses for scene {i+1} ({len(scene)} images)...")
        scene_poses = estimate_poses(scene, features_dict)
        
        # Count registered images
        registered_count = sum(1 for _, (R, T) in scene_poses.items() if R is not None and T is not None)
        print(f"  Registered {registered_count}/{len(scene)} images in scene {i+1}")
        
        all_poses.update(scene_poses)
        
        # Free memory
        free_memory()
    
    # Help garbage collection
    del features_dict
    del match_graph
    free_memory()
    
    return {
        'scenes': scenes,
        'outliers': outliers,
        'poses': all_poses
    }

def create_submission_file(test_dir, output_file, training_structure, test_dataset_info):
    """Process all test datasets and create submission file.
    
    Args:
        test_dir: Base directory containing all test datasets
        output_file: Path to output submission file
        training_structure: Structure information from training data
        test_dataset_info: Additional analysis info for test datasets
        
    Returns:
        Dataframe with submission results
    """
    # Initialize results list
    results = []
    
    # Check if test directory exists
    test_path = Path(test_dir)
    if not test_path.exists():
        print(f"Error: Test directory not found: {test_dir}")
        return pd.DataFrame()
    
    # Get list of test datasets
    test_datasets = [p for p in test_path.glob('*') if p.is_dir()]
    if not test_datasets:
        print(f"Error: No datasets found in {test_dir}")
        return pd.DataFrame()
    
    print(f"\nFound {len(test_datasets)} test datasets")
    
    # Process each test dataset
    for dataset_path in test_datasets:
        dataset_name = dataset_path.name
        print(f"\nProcessing test dataset: {dataset_name}")
        
        # Process dataset
        dataset_results = process_test_dataset(
            dataset_path, 
            dataset_name, 
            training_structure,
            test_dataset_info
        )
        
        # Add results to list
        # 1. Process scenes
        for scene_idx, scene in enumerate(dataset_results['scenes']):
            for img_name in scene:
                R, T = dataset_results['poses'].get(img_name, (None, None))
                
                results.append({
                    'dataset': dataset_name,
                    'scene': f"cluster{scene_idx+1}",
                    'image': img_name,
                    'rotation_matrix': format_matrix_for_submission(R),
                    'translation_vector': format_vector_for_submission(T)
                })
        
        # 2. Process outliers
        for img_name in dataset_results['outliers']:
            results.append({
                'dataset': dataset_name,
                'scene': "outliers",
                'image': img_name,
                'rotation_matrix': ";".join(["nan"] * 9),
                'translation_vector': ";".join(["nan"] * 3)
            })
        
        print(f"  Added {len(dataset_results['scenes'])} scenes and {len(dataset_results['outliers'])} outliers")
        
        # Clear memory after each dataset
        free_memory()
    
    # Convert to dataframe - do this in batches if the dataset is large
    if results:
        # Create dataframe in smaller chunks to save memory
        batch_size = 1000
        dfs = []
        for i in range(0, len(results), batch_size):
            batch = results[i:i+batch_size]
            dfs.append(pd.DataFrame(batch))
            
        results_df = pd.concat(dfs, ignore_index=True)
        del dfs  # Free memory
        free_memory()
    else:
        results_df = pd.DataFrame(columns=[
            'dataset', 'scene', 'image', 'rotation_matrix', 'translation_vector'
        ])
    
    # Ensure all test images are included in the submission
    all_test_images = {}
    for dataset_path in test_datasets:
        dataset_name = dataset_path.name
        image_files = [img.name for img in dataset_path.glob('*.png')]
        all_test_images[dataset_name] = set(image_files)
    
    # Check for missing images
    missing_images = []
    for dataset_name, images in all_test_images.items():
        if dataset_name in results_df['dataset'].values:
            submitted_images = set(results_df[results_df['dataset'] == dataset_name]['image'])
            missing = images - submitted_images
            if missing:
                print(f"Warning: Missing {len(missing)} images from dataset {dataset_name}")
                for img_name in missing:
                    missing_images.append({
                        'dataset': dataset_name,
                        'scene': "outliers",  # Assume missing images are outliers
                        'image': img_name,
                        'rotation_matrix': ";".join(["nan"] * 9),
                        'translation_vector': ";".join(["nan"] * 3)
                    })
        else:
            # No results for this dataset at all
            print(f"Warning: No results generated for dataset {dataset_name}")
            for img_name in images:
                missing_images.append({
                    'dataset': dataset_name,
                    'scene': "outliers",  # Assume all are outliers
                    'image': img_name,
                    'rotation_matrix': ";".join(["nan"] * 9),
                    'translation_vector': ";".join(["nan"] * 3)
                })
    
    # Add any missing images to results
    if missing_images:
        missing_df = pd.DataFrame(missing_images)
        if results_df.empty:
            results_df = missing_df
        else:
            results_df = pd.concat([results_df, missing_df], ignore_index=True)
        del missing_df  # Free memory
        free_memory()
    
    # Make sure we have all required columns
    if not results_df.empty:
        # Add image_id column
        results_df.insert(0, 'image_id', range(len(results_df)))
        
        # Save to CSV
        results_df.to_csv(output_file, index=False)
        print(f"\nSubmission file created: {output_file}")
        print(f"Total entries: {len(results_df)}")
    else:
        print("Error: No results generated. Submission file not created.")
    
    return results_df

## Generate Submission File

In [ ]:
# Execute the complete pipeline
submission_df = create_submission_file(TEST_DIR, OUTPUT_FILE, training_structure, test_dataset_info)

# Display a sample of the submission file
if not submission_df.empty:
    print("\nSubmission sample:")
    print(submission_df.head())
    
    # Verify the structure
    print("\nSubmission structure validation:")
    print(f"Total rows: {len(submission_df)}")
    print(f"Unique datasets: {submission_df['dataset'].nunique()}")
    print(f"Datasets: {', '.join(submission_df['dataset'].unique())}")
    
    # Check for required pattern in scene names
    scene_pattern_valid = all(s == 'outliers' or s.startswith('cluster') for s in submission_df['scene'].unique())
    print(f"Scene naming format valid: {scene_pattern_valid}")
    
    # Count scenes and outliers
    scene_counts = {}
    outlier_counts = {}
    
    for dataset in submission_df['dataset'].unique():
        dataset_df = submission_df[submission_df['dataset'] == dataset]
        
        # Count unique scenes (excluding outliers)
        scenes = dataset_df[dataset_df['scene'] != 'outliers']['scene'].unique()
        scene_counts[dataset] = len(scenes)
        
        # Count outliers
        outlier_counts[dataset] = len(dataset_df[dataset_df['scene'] == 'outliers'])
    
    print("\nScene and outlier counts:")
    for dataset, scene_count in scene_counts.items():
        print(f"  {dataset}: {scene_count} scenes, {outlier_counts[dataset]} outliers")
else:
    print("\nError: Submission file generation failed")